In [2]:
import json
import pandas as pd
import requests
import mysql.connector

# API Request

url = "https://api.sportradar.com/tennis/trial/v3/en/complexes.json?api_key=poXVLPnWV8ZufVVMyW2VkoXbjHjPVpF07nbAVPUL"
headers = {"accept": "application/json"}
response = requests.get(url, headers=headers)

# Load JSON response

data=response.json()

# Convert JSON to DataFrames

complexes = data.get("complexes", [])  


# Normalize JSON into DataFrames

df_complexes = pd.json_normalize(complexes) 
venues = [venue | {"complex_id":complex['id']} for complex in complexes for venue in complex.get("venues", [])]
df_venues = pd.json_normalize(venues) 

# Connect to MySQL Database

mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="NanDha@12345",
    database="sports_radar_api",
    auth_plugin='mysql_native_password'
)
cursor = mydb.cursor()

# Insert into Complexes Table (if DataFrame is not empty)

for index, row in df_complexes.iterrows():
            sql = """
            INSERT INTO complexes (complex_id, complex_name) 
            VALUES (%s, %s) 
            ON DUPLICATE KEY UPDATE complex_name = VALUES(complex_name)
            """
            values = (row.get('id'), row.get('name'))
            cursor.execute(sql, values)

# Insert into Venues Table (if DataFrame is not empty)
for index, row in df_venues.iterrows():
            
            sql = """
            INSERT INTO venues (venue_id, venue_name, city_name, country_name, country_code, timezone, complex_id)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
            ON DUPLICATE KEY UPDATE 
                venue_name = VALUES(venue_name), 
                city_name = VALUES(city_name), 
                country_name = VALUES(country_name), 
                country_code = VALUES(country_code), 
                timezone = VALUES(timezone), 
                complex_id = VALUES(complex_id)
            """
            values = (
                row.get('id'), row.get('name'), row.get('city_name'), row.get('country_name'),
                row.get('country_code'), row.get('timezone'), row.get('complex_id')
            )
            cursor.execute(sql, values)

# Commit Changes & Close Connection

mydb.commit()
cursor.close()
mydb.close()